<a href="https://colab.research.google.com/github/ShadowofSkull/DevHack2023/blob/main/gcolab/featureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!python -m pip install tensorflow
!python -m pip install transformers datasets
!python -m pip install pandas
!pip install -U sentence-transformers
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transf

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd

df = pd.read_csv("drive/MyDrive/Colab Notebooks/data/Resume.csv")
resume = Dataset.from_pandas(df)
job = load_dataset("jacob-hugging-face/job-descriptions")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from datasets import get_dataset_split_names

get_dataset_split_names("jacob-hugging-face/job-descriptions")


['train']

In [4]:
print(job, resume)

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
}) Dataset({
    features: ['ID', 'Resume_str', 'Resume_html', 'Category'],
    num_rows: 2484
})


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
from spacy.matcher import Matcher


In [43]:
nlp = spacy.load('en_core_web_sm')

# By finding frequency of nouns occuring to determine which sentences have more importance
def summariser(text):
  text = text.strip().lower().replace("\n", "")
  doc = nlp(text)

  # To get nouns
  matcher = Matcher(nlp.vocab)
  pattern = [
      {"POS": "PROPN", "OP": "+"},
      {"POS": "NOUN", "OP": "+"},

  ]
  pattern1 = [
      {"POS": "NOUN", "OP": "+"},
      {"POS": "PROPN", "OP": "+"},

  ]
  pattern2 = [
      {"POS": "NOUN", "OP": "+"},

  ]
  pattern3 = [
      {"POS": "PROPN", "OP": "+"},

  ]
  matcher.add("label", [pattern, pattern1, pattern2, pattern3], greedy="LONGEST")
  matches = matcher(doc)
  # To sort in the order of occurance in the resume
  matches.sort(key = lambda x: x[1])
  # print(len(matches))
  keyword = []
  stopwords = list(STOP_WORDS)

  # remove stopwords and punctuation
  for mat in matches:
    text = doc[mat[1]: mat[2]]
    if not (text.text in stopwords or text.text in punctuation):
      keyword.append(text.text)
      # print(mat, text.text)
  # print(keyword)

  freq_word = Counter(keyword)
  # print(freq_word.most_common())
  max_freq = Counter(keyword).most_common(1)[0][1]
  for word in freq_word.keys():
        freq_word[word] = (freq_word[word]/max_freq)

  print(freq_word.most_common(20))

  sent_strength={}
  for sent in doc.sents:
      for word in sent:
          if word.text in freq_word.keys():
              if sent in sent_strength.keys():
                  sent_strength[sent]+=freq_word[word.text]
              else:
                  sent_strength[sent]=freq_word[word.text]
  # print(sent_strength)

  summarized_sentences = nlargest(10, sent_strength, key=sent_strength.get)
  # print(summarized_sentences)

  final_sentences = [ w.text for w in summarized_sentences ]
  summary = ' '.join(final_sentences)
  return(summary)

In [54]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('BAAI/bge-large-en-v1.5')

res = """
         WEB DESIGNER/GRAPHIC DESIGNER       Summary    To obtain a dynamic, challenging opportunity that contributes to the success of the business via eight years proposal coordination experience and 15 years' experience in graphic layout and design. Make a strong contribution by utilizing and expanding upon work experiences and capabilities, related education, oral, written and interpersonal skills. Positively impact the quality of work, acquisition of new business and client retention goals.      Highlights        Proficient in Dreamweaver, Photoshop, QuarkXPress, InDesign, Microsoft Word and PowerPoint; Foundational skills in Illustrator and Microsoft Excel
Copyediting, proofreading, HTML and CSS editing
In depth knowledge of Business Development and Proposal response and coordination process              Experience     01/2010       Web Designer/Graphic Designer    Company Name   ï¼   City  ,   State      Manufacturer of Expansion Joints and Hose Manage front-end web site design to project a visually appealing image of company products and value added services.  Manage all assigned projects from conceptualization to implementation.  Coordinating and communicating with stakeholders in all stages of work to achieve desired message and impact.  Update existing site content through collaboration with subject matter experts to ensure industry relevancy and accuracy.  Develop and redesign company catalogs, periodic internal publications, industry advertisements and sell sheets in a manner that will project a sophisticated image and result in additional sales revenue for the company.  Overhauled the visual appeal and readability of previously developed technical and product catalogs through extensive redesigning.         01/2010   to   Current     Web, Graphics and Written Communications Consultant      Family In Distress, Inc., Augustine Monica Films, LaGrande Famille Preschool, Black Pearl, Inc.  Web site design and maintenance for a variety of clients to provide or enhance online presence, as well as promote the sale of products and services.  Provide graphic design support for non-profit, small business and individual clients to project a professional and polished print image to target audiences.  Written business communications support by gathering information and effectively translating verbal dialogue into well-written, persuasive copy.         01/2008   to   01/2012     Copy Editor and Proofreader    Company Name   ï¼   City  ,   State      Thoroughly proofread web-based documents for The Academy of Nutrition and Dietetics Evidence Analysis Library www.eatright.org (the world's largest organization of food and nutrition professionals).  Copyedited nutritional research studies for accuracy according to ADA's guidelines including citation, terminology and specified table formatting for consistency throughout the extensive user-friendly online library.  Funneled documents to lead editors for final publication and online availability to dietetic and nutrition professionals, researchers and advocates worldwide.         01/1999   to   01/2007     Proposal Specialist    Company Name   ï¼   City  ,   State      Provided professional proposal development support to all divisions of the ARAMARK Corporation.  Achievements Received "2006 Top Producer" Award for overall annual production and outstanding peer review.  Recognized as "Quarterly Top Producer" several quarters over the course of eight-year tenure at ARAMARK.  Recipient of above average annual merit raises as a result of outstanding performance reviews.  Key Responsibilities: Executed analysis and preparation of RFP'S/RFQ'S/RFI'S in coordination with ARAMARK business development team professionals.  Ensured responses were completed and received operating in a fast-paced, deadline-driven and time-sensitive environment.  Coordinated with ARAMARK company resources to develop business winning, quality customized responses to RFP/RFQ/RFI's including legal, compliance and finance departments as well lines of service and subject matter experts.  Applied ARAMARK corporate methodology and quality assurance procedures to ensure unified branded message and image were delivered every time.  Implemented use of standard corporate response sections based on boilerplate.  Supported customization through editing and/or writing when necessary.  Performed editing and proofreading to ensure accuracy and integrity of all proposal documentation including content flow and organization, syntax and grammar.  Created final documents utilizing QuarkXPress, Photoshop, PowerPoint and Excel.  Assisted in the development and maintenance of database/repository of past responses and supplemental documents including client references, key personnel bios, and rÃ©sumÃ©s.  Supported efforts of directors of business development in procuring new business and retaining existing clients through creation of engaging, informative and well-organized marketing materials, annual client progress reports and case studies.  Regularly communicated progress status during the life cycle of an RFP; kept director of business development abreast of relevant project information concerning addenda or changes to initial response specifications.  Served as a valuable company resource by developing and maintaining a thorough knowledge of ARAMARK's extensive lines of business and services offered.  Documented valuable lessons learned to share with proposal manager and department team members.  Implemented newfound efficiencies and/or best practices into response development process.  Served as a company resource by continuously developing graphic design and layout skills to present sales material in a visually appealing and engaging format.          Education     May 1991     Bachelor of Science  :   Journalism    Boston University College of Communication          Journalism       February 2007     Graphic Design Foundation Certificate      Sessions School of Design                      Certificate in Web Development - February 2002      Illinois Institute of Art                  Skills    ADA, Photoshop, advertisements, business communications, business development, catalogs, content, Copyediting, CSS, client, clients, database, documentation, Dreamweaver, editing, fast, finance, Graphic Design, HTML, Illustrator, image, InDesign, layout, legal, director, marketing materials, Microsoft Excel, Excel, PowerPoint, Microsoft Word, performance reviews, Pearl, personnel, persuasive, Producer, profit, progress, proofreading, Proposal, proposal development, publications, publication, quality, quality assurance, QuarkXPress, research, RFI, RFP, sales, translating, Web Development, Web site design, well-organized, Written
"""

res = res.strip().lower()
res = summariser(res)

job_descs = []
resumes = []
# for jb in job["train"]:
for i in range(25):
  sum_des = summariser(job["train"][i]["job_description"])
  job_descs.append(sum_des)
  resumes.append(res)

#Compute embedding for both lists
embeddings1 = model.encode(job_descs, normalize_embeddings=True)
embeddings2 = model.encode(resumes, normalize_embeddings=True)
# embeddings1 = model.encode(sentences1, convert_to_tensor=True)
# embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(job_descs)):
    # print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
    print(f"Score: {cosine_scores[i][i]*100:.2f}, Index: {i}")


Score: 52.83, Index: 0
Score: 51.70, Index: 1
Score: 52.35, Index: 2
Score: 68.46, Index: 3
Score: 59.13, Index: 4
Score: 60.32, Index: 5
Score: 72.69, Index: 6
Score: 69.13, Index: 7
Score: 63.05, Index: 8
Score: 66.70, Index: 9
Score: 62.75, Index: 10
Score: 58.65, Index: 11
Score: 63.80, Index: 12
Score: 63.11, Index: 13
Score: 72.20, Index: 14
Score: 64.97, Index: 15
Score: 49.62, Index: 16
Score: 53.00, Index: 17
Score: 52.15, Index: 18
Score: 47.45, Index: 19
Score: 55.31, Index: 20
Score: 55.78, Index: 21
Score: 47.49, Index: 22
Score: 53.90, Index: 23
Score: 48.60, Index: 24


In [85]:
print(model.get_max_seq_length())
print(len(res))
print(type(cosine_scores))
# have split a doc into paragraph of 512 token or use text summariser instead

512
3556
<class 'torch.Tensor'>


In [75]:
scores = []
for i in range(25):
  scores.append(str(cosine_scores[i][i])[7:13])

top_scores = nlargest(5, scores)
best_jobs = []
for i in range(5):
  idx = scores.index(top_scores[i])
  best_jobs.append([top_scores[i], idx])

print(best_jobs)


[['0.7269', 6], ['0.7220', 14], ['0.6913', 7], ['0.6846', 3], ['0.6670', 9]]


In [84]:
for i in range(5):
  title = job["train"][best_jobs[i][1]]["position_title"]
  name = job["train"][best_jobs[i][1]]["company_name"]
  similarity = float(best_jobs[i][0]) * 100
  print(f"#{i + 1} Job position is {title} for {name} company with {similarity:.2f}% similarity rate")
  print("When it's 60 and above its usually decently accurate")

#1 Job position is Remote Website Designer for Equisolve, Inc. company with 72.69% similarity rate
#2 Job position is UI Web Designer for Western Governors University company with 72.20% similarity rate
#3 Job position is Web Designer for Zander Insurance Agency company with 69.13% similarity rate
#4 Job position is Web Designer for Robert Half company with 68.46% similarity rate
#5 Job position is SR. Web Designer for General Dynamics Information Technology company with 66.70% similarity rate
